# Image data - 5-way classifier
# Stratified KFold implementation. 

In [ ]:
!pip install shap

In [ ]:
pip install matplotlib  numpy  pandas scipy  seaborn sinfo sklearn  tensorflow umap-learn notebook

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm as tqdm
import random
import seaborn as sns
import os
import glob
import tensorflow as tf
from sklearn.metrics import classification_report, roc_curve, auc, accuracy_score
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss

from PIL import Image
import shap
%matplotlib inline
from PIL import Image

# set results paths

In [ ]:
path_res = Path("/project/restored-files/DiseaseClassifierPaper2021/chemPredPD/demo_output/image_data_5way_skfold")

In [ ]:
#code to set the path to save the model
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

In [ ]:
#code to set the path to save plots
path_plots = path_res.joinpath("plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
#code to create folders if not present
if not path_models.exists():
    path_models.mkdir(parents=True, exist_ok=True)
path_models.is_dir()
if not path_plots.is_dir():
    path_plots.mkdir(parents=True, exist_ok=True)
path_plots.is_dir()

# load images

In [ ]:
directory = Path("/project/restored-files/DiseaseClassifierPaper2021/chemPredPD/demo_data/Image_15000/")
#directory = Path('/tmp/Image_data/')
subdirectory = list(directory.glob('*'))
print(subdirectory)

In [ ]:
#Paths for different PD subtypes 
SNCAx3_paths = glob.glob(str(directory)+"/**/*3xSNCA*", recursive=True)
oligomer_paths =glob.glob(str(directory)+"/**/*oligomer*", recursive=True)
rotenone_paths =glob.glob(str(directory)+"/**/*rotenone*", recursive=True)
oligomycin_paths =glob.glob(str(directory)+"/**/*oligomycin*", recursive=True)
control_paths = glob.glob(str(directory)+"/**/*control*", recursive=True)
untreated_paths = glob.glob(str(directory)+"/**/*untreated*", recursive=True)

In [ ]:
def sample_function(): 
    return 0.5

In [ ]:
all_paths = SNCAx3_paths + oligomer_paths + rotenone_paths + oligomycin_paths + control_paths + untreated_paths
random.shuffle(all_paths, sample_function)

In [ ]:
print(len(SNCAx3_paths))
print(len(oligomer_paths))
print(len(rotenone_paths))
print(len(oligomycin_paths))
print(len(control_paths))
print(len(untreated_paths))

In [ ]:
len(all_paths)

In [ ]:
random.shuffle(all_paths)

In [ ]:
all_paths[199]

In [ ]:
Image.open(all_paths[788])

In [ ]:
def label_from_path(path):
    path = str(path)
    if 'control' in path:
        return 0
    elif 'untreated' in path:
        return 0
    elif '3xSNCA' in path:
        return 1
    elif 'oligomer' in path:
        return 2
    elif 'rotenone' in path:
        return 3
    elif 'oligomycin' in path:
        return 4
    else:
        raise Exception(
            "not disease"
        )

In [ ]:
data_paths = np.array(all_paths)
nb_classes = 5
labels = np.array([label_from_path(path) for path in data_paths]).reshape(-1)

In [ ]:
def _load_image(path):
    raw_data = tf.io.read_file(path)
    integer_pixels = tf.image.decode_jpeg(raw_data, channels=3)
    return tf.image.convert_image_dtype(integer_pixels, dtype=tf.float32)

def build_dataset(paths, labels, batch_size=100, shuffle=True):
    resize_x = 84
    resize_y = 84
    ds = tf.data.Dataset.from_tensor_slices(
        (tf.stack(paths), tf.stack(labels))
    )
    ds = ds.map(lambda x, y: (_load_image(x), y))
    ds = ds.map(
        lambda x, y: (tf.image.resize(x, [resize_x, resize_y]), y)
    )
    if shuffle is True:
        ds = ds.shuffle(3*batch_size)
    ds = ds.batch(batch_size)
    ds = ds.cache()
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    ds = ds.repeat()
    return ds

In [ ]:
def build_test_dataset(paths, labels, batch_size=100):
    resize_x = 84
    resize_y = 84
    ds = tf.data.Dataset.from_tensor_slices(
        (tf.stack(paths), tf.stack(labels))
    )
    ds = ds.map(lambda x, y: (_load_image(x), y))
    ds = ds.map(
        lambda x, y: (tf.image.resize(x, [resize_x, resize_y]), y)
    )
    ds = ds.batch(batch_size)
    ds = ds.cache()
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
def evaluate_model(images, labels, n_folds, path_to_save_models):
    #n_folds=2
    #path_to_save_models = path_models

    X = data_paths
    y = labels 

    labels_one_hot = tf.keras.utils.to_categorical(labels)
    scores, histories = list(), list()
    fold_ct = 0

    #tprs, aucs = list(), list()
    results = list()

    # prepare cross validation
    skf = StratifiedKFold(n_splits= n_folds, shuffle=True, random_state=5)
    for train_index, test_index in skf.split(X, y):
        fold_ct+=1
        print("Fold", fold_ct)
        train_val_images, test_images = images[train_index], images[test_index]
        train_val_labels_one_hot, test_labels = labels_one_hot[train_index] , labels_one_hot[test_index]

        train_images, val_images, train_labels, val_labels = train_test_split(
                    train_val_images, train_val_labels_one_hot, random_state=42,stratify= train_val_labels_one_hot
                        )
        
        train_val_paths, test_paths = data_paths[train_index], data_paths[test_index]
        train_val_labels_one_hot, test_labels = labels_one_hot[train_index] , labels_one_hot[test_index]

        train_paths, val_paths, train_labels, val_labels = train_test_split(
                    train_val_paths, train_val_labels_one_hot, random_state=42,stratify= train_val_labels_one_hot
                        )

        print("train_paths ", train_paths.shape)
        print("val_paths.shape", val_paths.shape)
        print("test_paths.shape", test_paths.shape)


        #Building the datasets
        #Here the val is shuffled in line with the train dataset
        batch_size = 256
        ds_train = build_dataset(train_paths, train_labels, batch_size=batch_size)
        ds_val = build_dataset(test_paths, test_labels, batch_size=batch_size)
        ds_test = build_test_dataset(test_paths, test_labels, batch_size=batch_size)
        


        #Neural Net Architecture

        model = Sequential()
        model.add(
        Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu', input_shape=(84, 84, 3))
        )
        model.add(
        MaxPooling2D()
                 )
        model.add(
            Dropout(0.2)
        )
        model.add(
        Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu')
        )
        model.add(
        MaxPooling2D()
                 )
        model.add(
            Dropout(0.2)
        )
        model.add(
        Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu')
        )
        model.add(
        MaxPooling2D()
                 )
        model.add(
            Dropout(0.2)
        )
        model.add(
        Conv2D(filters=32, kernel_size=(3,3), strides=1, activation='relu')
        )
        model.add(
        MaxPooling2D()
                 )
        model.add(
            Dropout(0.2)
        )
        model.add(
        Flatten()
        )
        model.add(
            Dense(10, activation="relu"))
        model.add(
            Dense(5, activation="softmax"))
        
        model.summary()
        
        
        #Training
        model_name = f"5-class_fold{fold_ct}" 

        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            min_delta=0, 
            patience=50, 
            verbose=0, 
            mode='auto',
            baseline=None, 
            restore_best_weights=True,
        )

        checkpoint = ModelCheckpoint(
            str(model_name)+'.hdf5', 
            monitor='val_loss', 
            verbose=1,
            save_best_only=True, 
            mode='auto', 
            save_freq='epoch'
        )
        
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        
        model.compile(
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer=optimizer
        )

        epochs = 1000

        history = model.fit(
            ds_train,
            validation_data=ds_val,
           steps_per_epoch=len(train_paths) // batch_size,
            validation_steps=len(test_paths) // batch_size,
            epochs=epochs,
            callbacks=[early_stop,checkpoint]
        )

        ## Load the Best Model
        model_saved = load_model(str(model_name)+'.hdf5')

        ## Inference & Performance
        y_pred = model.predict(
            ds_test,
            steps=(len(test_paths) // batch_size) + 1,
        )
        
        acc = accuracy_score(test_labels.argmax(axis=1), y_pred.argmax(axis=1))
        print('> %.3f' % (acc * 100.0))
        # stores scores
        #scores.append(acc)
        #for roc score
        results.append([acc])
        histories.append(history)


        del model_saved, model, ds_val, ds_train, ds_test
    res = pd.DataFrame( results, columns= ['accuracy'])
    return histories, res

In [ ]:
def summarize_diagnostics(histories, title, path_save_plot, img_dpi=300 ):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))
    for i in range(len(histories)):
        ax1.plot(histories[i].history['loss'], color='blue',  label='Training')
        ax1.plot(histories[i].history['val_loss'], color='orange', label='Validation')
    
        ax2.plot(histories[i].history['accuracy'], color='blue', label='Training')
        ax2.plot(histories[i].history['val_accuracy'], color='orange', label='Validation')
    
    ax1.set_title("Loss")
    ax1.set_ylabel("Loss")
    ax1.set_xlabel("Epochs")
    ax1.legend(frameon=False, labels = ["Training", "Validation"])
    
    ax2.set_title("Accuracy")
    ax2.set_ylabel("Accuracy")
    ax2.set_xlabel("Epochs")
    ax2.legend(frameon=False, labels = ["Training", "Validation"])

    fig.suptitle(title);
    fig.savefig(path_save_plot.joinpath("image_field_8x8_healthy_disease_Model-perform.png"), bbox_inches='tight', dpi = img_dpi)
    
    return 

In [ ]:
def summarize_scores(scores, title ):
    #print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(scores)*100, np.std(scores)*100, len(scores)))
    # box and whisker plots of results
    plt.boxplot(scores)
    plt.suptitle(title)
    plt.show()
    return

In [ ]:
histories, results = evaluate_model(data_paths, labels, n_folds = 10, path_to_save_models =path_models)

# Plot model histories and scores

In [ ]:
results.to_pickle(path_models.joinpath("image_disease-healthy_accuracy_auc_scores.pkl")) 

In [ ]:
plot_title = "Image data - 5 class model, ssk-fold"
summarize_diagnostics(histories, title=plot_title, path_save_plot = path_plots)

In [ ]:
results

In [ ]:
plot_title = ("Image data - 5 class model, ssk-fold,"
                              "\n Accuracy")
summarize_scores(results, plot_title )